# Mandarin dictionnary


On going 


In [75]:
import pickle
import re
import codecs

from docx import Document

import jieba as ji
import jieba.analyse as ja

__dict_path = '/Users/macbuse/jieba-master/extra_dict/'
ja.set_stop_words(__dict_path + "stop_words.txt")
ja.set_idf_path(__dict_path + "../extra_dict/idf.txt.big");

__dict_path = '/Users/macbuse/jieba-master/extra_dict'

ep = re.compile('a|e|i|o|u')

#I took the html stuff and now it's a string in this module
from html_code import *

In [115]:
class MyWords(dict):
    
    def __init__(self,  
                 src_pkl='words.pkl',
                 tooltip='test'):
        self.ordered_keys = []
        self.src = src_pkl
        tail = ' data-placement="top" title="%s">%s </a>'
        self.wrapper = '<a class="%s" href="#" data-toggle="tooltip"'%tooltip + tail 
        
    
    def tidy(self):
        LL = self.keys()
        LL.sort()
        self.ordered_keys = LL
        
    def show_all(self):
        self.tidy()
        for x in self.ordered_keys:
            print self.show(x)
            
    def look_up(self, other):
        return [self.show(x) for x in other if x in self]
    
    def wrap(self, entity):
        
        if entity in self and self[entity]: 
            payload = '%s : %s' %self[entity]
            if isinstance(payload, str):
                payload = unicode(payload,'utf8')
            return self.wrapper%(payload, entity)
        else:
            return entity
         
    def missing(self, other):
        return [x for x in other if x not in self]
    
    def load(self):
        tt = pickle.load(file(self.src,'r') ) 
        self.update(tt)
        
    def dump(self):
        #not nice but we want to get it back one day
        #even if we have modified the class definition so...
        copy = {}
        copy.update(self)
        with file(self.src, 'wb') as fp:
            pickle.dump(copy, fp)
         
    def show(self, key):
        if not self[key]: return key + ' None'
        payload = ' %s : %s' %self[key]
        if isinstance(payload, str):
            payload = unicode(' %s : %s' %self[key], 'utf8')
        return key + payload
    
def txt2web(paras=['',''],
            my_words=[MyWords()],
            fn='test_tool.html'):
    
    '''mark up a text with tool tips
    from a list of  MyWord objects'''
    import html_code
    my_html = []
    for para in paras[:]:
        txt = [x[0] for x in ji.tokenize(para)]
        for word_list in my_words:
            #wrap() returns the string 
            #if it has already been  treated
            txt = [word_list.wrap(w) for w in txt]              
        my_html.append(txt)

    my_html = [ '<p>%s</p>\n'%' '.join( para) for para in html]
    my_html = ''.join( my_html)
    my_html= html_wrapper%my_html
    
    with codecs.open( fn, 'w', encoding='utf8') as ff:
        ff.write( my_html)

In [118]:
txt2web(paras, my_words=[wiki_words,tt])

In [93]:
tt = MyWords(src_pkl='m2e_word_pinyin_meaning.pkl',
             tooltip='test1')
tt.load()

In [ ]:
doc = Document('/Users/macbuse/Downloads/ZHU_TRANSLATIONS/holler_CN.docx')
doc = Document('/Users/macbuse/Downloads/ZHU_TRANSLATIONS/汪化 灵魂深处的生长 .docx')

In [54]:
for x in doc.paragraphs[:1]:
    print x.text
    print '-'*60

汪化 灵魂深处的生长 Growth from depth of the soul
------------------------------------------------------------


In [396]:
result = ji.tokenize(doc.paragraphs[2].text)
for tk in list(result)[:10]:
    print u"%s\t" % tk[0],

在	早期	的	艺术作品	中	，	你	显示	对	数学	

In [96]:
#content = '\n'.join( [para.text for para in doc.paragraphs])

def get_words(txt):
    topK = 100
    tags = ja.extract_tags(txt, topK=topK)
    # if it is a number or english then kill it
    p_kill = re.compile('(\d+)|(a|e|i|o|u)')
    #always convert to lower
    return [x for x in tags if not p_kill.search(x.lower()) ]

top100 = get_words(' '.join(contents2))

In [102]:
mm = wiki_words.missing(top100) 
print '\ '.join( tt.missing(mm) )

张帆\ 绘画\ 布面\ 劳作\ 急就章\ 自叙\ 属于\ 成为\ 山林\ 现在\ 逻辑\ 危险\ 视为\ 奇观\ 气息\ 基座\ 形式主义\ 水图\ 抽象画\ 一张\ 书写\ 丙烯\ 质感


In [412]:
tokens = get_words(' '.join(paras) )
for x in tt.look_up(tokens):
    print x

一种 yī zhǒng : one kind of, one type of
这个 zhè ge : this, this one
方式 fāng shì : way, manner, style, mode, pattern, 
这种 zhè zhǒng : this, this kind of, this sort of, this type of
系列 xì liè : series, set
过程 guò chéng : course of events, process, 
抽象 chōu xiàng : abstract, abstraction
一个 xià yī ge : the next one
我们 wǒ men : we, us, ourselves, our
发展 fā zhǎn : development, growth, to develop, to grow, to expand
自我 zì wǒ : self-, ego (psychology)
作品 zuò pǐn : work (of art), opus, 
所以 suǒ yǐ : therefore, as a result, so, the reason why
中国 Zhōng guó : China
线条 xiàn tiáo : line (in drawing, calligraphy etc), the lines or contours of a three-dimensional object (hairstyle, clothing, car etc)
这些 zhè xiē : these
只有 zhǐ yǒu : only
可能 kě néng : might (happen), possible, probable, possibility, probability, maybe, perhaps
展览 zhǎn lǎn : to put on display, to exhibit, exhibition, show, 
传统 chuán tǒng : tradition, traditional, convention, conventional
早期 zǎo qī : early period, early phase, early stage
当代

In [414]:
print ' '.join(tt.missing(tokens) )

张帆 绘画 布面 劳作 急就章 自叙 属于 成为 山林 现在 逻辑 危险 视为 奇观 基座 气息 仍然 形式主义 水图 抽象画 一张 书写 丙烯 质感 油画 采薇 符号 缓慢 似乎 高度 引自 僵尸 命题 苏轼 废品 存在 几个 有着 古典 几乎 真的 境界 身边 某个 容易 变化 格子 今格 设想 还是 消失 国际 画面 摹拟 之三 它们 完成 占据 禁锢


# Data extractors

### Sources for vocabulary : 

1. https://en.wiktionary.org/wiki/
1. https://chinese.yabla.com

Neither of these are difficult to scrape

#### Test texts :

1. art.china.cn
1. mp.weixin.qq.com

Neither of these are difficult to scrape
but they use different container classes...see code


### Wiki

The big problem here is that there are repeated <a></a> tags
and the regexp isn't trivial

`ur'<a.*?>(.*?)</a></span>.*?<a.*?</a></span>.*?<a.*?>(.*?)</a>\)(.*?)<' `

### Yabla

This is a standard data collector :

1. should check the current dictionnaries before making a request
2. the utf8 has to be encoded for the request string



In [106]:
import urllib2

def yabla_lookup(word_bag = []):
    pdef =  re.compile(ur'class="definition">.*?="pinyin">(.*?)<.*?="meaning">(.*?)<',re.DOTALL)
    stuff = {x:'' for x in word_bag} 

    for word in word_bag:
        print word,
        qq = urllib2.quote(word.encode('utf8'))
        base_url = u'https://chinese.yabla.com/chinese-english-pinyin-dictionary.php?define='
        data = urllib2.urlopen(base_url + qq).read()
        stuff[word] = pdef.search(data)
    return stuff

def clean(txt, meaning=False):
    if meaning : return txt.split('\n')[0] + ' '
    pinyin, meaning = y.group(1), y.group(2)
    meanings = meaning.split('CL:')[0] 
    #there is a trailing \n so strip() it 
    return (pinyin, meaning.strip().replace('\n',', ') )

In [ ]:
import urllib2, re

dp = ur'<a.*?>(.*?)</a></span>.*?<a.*?</a></span>.*?<a.*?>(.*?)</a>\)(.*?)<'
dp = re.compile(dp)

base_url = 'https://en.wiktionary.org/wiki/Appendix:Mandarin_Frequency_lists'

n = 4
data = urllib2.urlopen(base_url + '/%d001-%d000'%(n,n+1)).read()
matches = dp.findall(data)

for x in matches[:]:
    entity, pinyin, meaning = [unicode(w,'utf8') for w in x]
    wiki_words[entity = (pinyin, meaning[3:])

In [ ]:
from bs4 import BeautifulSoup
import urllib2

#data = urllib2.urlopen('http://art.china.cn/tongzhi/2018-03/13/content_40249473.htm').read()

#data = urllib2.urlopen('http://art.china.cn/ysjt/2017-09/20/content_40022118.htm').read()
#data = urllib2.urlopen('http://art.china.cn/haiwai/2016-05/06/content_8753835.htm').read()
data = urllib2.urlopen('http://mp.weixin.qq.com/s/FIBpmy2GFPRMvXFNCd8lLg').read()

def extract_contents(html,
                        div_class='content'):
    
    '''return all the paragraphs in the html'''
    
    soup = BeautifulSoup(html, 'lxml')
    payload = soup.find('div', {'class' : div_class})
    return [p.text for p in payload.find_all('p')]  

class4 = {'art.china.cn': 'content', 'mp.weixin.qq.com' : "rich_media_content " }
contents2 = extract_contents(data, 
                             div_class=class4['mp.weixin.qq.com'])

In [110]:
tt.update(new_stuff)
tt.dump()
mm = wiki_words.missing(top100) 
print '\ '.join( tt.missing(mm) )
new_stuff = yabla_lookup(tt.missing(mm)) 

In [108]:
failed = []
for x,y in new_stuff.items():
    try:
        new_stuff[x] = clean(y)
    except:
        failed.append(x)
        
print '\n'.join(failed)
    

山林
一张
布面
急就章
水图
张帆
自叙
抽象画


So with what is left we are going to have to do some **mechanical turk**

Trouble is that google translate uses seriously obfuscated javascript....

In [109]:
src = u'''山林
采薇
引自
一张
真的
布面
今格
水图
自叙
抽象画'''

pyn = u'''Shānlín
cǎi wēi
yǐn zì
yī zhāng
zhēn de
bù miàn
jīn gé
shuǐ tú
zìxù
chōuxiàng huà'''

tgt = '''Mountain forest
Petch
cited from
One sheet
Really
Cloth surface
In this case
Water map
Self-description
Abstract painting'''

payload = zip([ x.lower() for x in pyn.split('\n') ],
[ x.lower() for x in tgt.split('\n') ])
for w, dd in zip( src.split('\n'), payload):
    new_stuff[w] = dd


Keep the dictionnary clean !! 
There is a risk of getting str from the clipboard 

In [419]:
for x in new_stuff.keys():
    if isinstance(x,str):
        del stuff[x]

for x in [u'苏摩 苏摩']:
    if x in new_stuff:
        del new_stuff[x]

In [76]:
for x,y in new_stuff.items():
    print x, '%s %s' %y

NameError: name 'new_stuff' is not defined

In [25]:
def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))
    def one_xlat(match):
        return adict[match.group(0)]
    return rx.sub(one_xlat, text)

In [248]:
#multiple_replace(content, m2ex)

In [49]:
content2 = '\n'.join(x.text for x in doc.paragraphs 
                             if x.text and len(ep.findall(x.text) ) < 30 )
#print content2

[ len(ep.findall(x.text) )for x in doc.paragraphs  if x.text] 


[7, 0, 20, 390, 0, 222, 5, 275, 355, 0, 186, 5]

In [52]:
[ len(ep.findall(x.text)) / float(len(x.text)) for x in doc.paragraphs  if x.text]

[0.175,
 0.0,
 0.055865921787709494,
 0.31630170316301703,
 0.0,
 0.2817258883248731,
 0.01845018450184502,
 0.29411764705882354,
 0.281076801266825,
 0.0,
 0.2957074721780604,
 0.18518518518518517]

In [89]:
paras = [ x for x in contents2 if len(x)>0 and len(ep.findall(x.lower()))/float(len(x)) <.2 ]

In [90]:
txt = '\n'.join(paras)
print txt
    

“张帆” 今格空间 展览现场
张帆的《急就章》是一组对现在说“不”的作品，这个“不”的核心，在于他认为现在的抽象画依赖劳作量赋予意义，或者把劳作量置换成一种观念，“靠巨大的劳作量和劳作时间来完成一张作品的这种方式属于奇观式的绘画”，“理论上说，这种绘画任何人都可以完成，只需要对一个物体反复进行一种简单机械的操作就可以成就一种奇观”，这些奇观企图掩盖这样一个事实：精神性逐渐丧失，抽象画已沦为平面构成和工业设计的玩物。
 
仔细想一想我们身边的状况，似乎恰如他反对的那样，几乎每天都有大量的抽象作品被生产出来，它们拥有称得上精耕细作的外观和质感，特定的个人符号和图式，即使在后一方面不那么突出，但通过长期的经营，终得以在某个阶段固执地占据了我们的记忆，当那一天到来的时候，这个画家就宣告成立了——这种方法论不止在我们身边生效，也在国际通行；“僵尸形式主义”这个术语的出现，正是对那些“把基于过程的抽象画搞得如此让人难以忍受的”艺术家的明确反讽。[1]
张帆，《急就章第六组之三》，布面丙烯，200x200cmx2，2017
不过，仅仅说“不”是容易的，也是危险的，危险在于：当你置身于一种对立性的逻辑中，自我有可能随对立面一起缩减。将这种危险阐释得更具体一些：在你锐化社会学或艺术现象学的某些命题的同时，你的思维和表达很容易会被圈禁其中，这是一种反向的禁锢，但仍然是禁锢，它提供了对抗的激情，但并不保证一个主体进行真正的自我建构——幸而，他不只是对现在说“不”，也是在对过去的自己说“不”，并且，他赖以说“不”的基座是我们渴望赎回的传统。
 
对于抽象艺术，张帆曾经抱有某种现代主义式的骄傲，他早期的格子系列耽留于那种经典框架内，以追随巴尼特·纽曼、罗斯科等人的方式，通过垂直的结构和优良的质地，来体现他“对庄重，崇高，含蓄的绘画追求”。设想他一直循沿这种做法，纵然会错过一、两次抽象在中国的热潮，也可以缓慢地占据一席之地，成为重要展览或收藏拼图的一角，但是，当他的认知产生变化之后，他几乎在另启炉灶，这并非缺乏耐心的表现，也不应该被视为一幕针对他人成功的复仇剧，原因只有一个：基座的重置——他想要在中国画传统中发掘抽象的可能。
 
张帆，《覆盖在白色，黑色表面的灰色以及棕色的习作之二》，布面油画，400x250cm，2009
《溪山林泉》和《水图》系列展示了他如何摒弃愉悦视觉的装饰感，向古典回溯

In [241]:
new_words = get_words(contents2)
for x in tt.look_up(new_words)[:10]:
    print x


观众 guān zhòng : spectators, audience, visitors (to an exhibition etc)
装置 zhuāng zhì : to install, installation, equipment, system, unit, device
一个 xià yī ge : the next one
作品 zuò pǐn : work (of art), opus, 
空间 kōng jiān : space, 
展览 zhǎn lǎn : to put on display, to exhibit, exhibition, show, 
选择 xuǎn zé : to select, to pick, choice, option, alternative
艺术家 yì shù jiā : artist, 
自然 zì rán : nature, natural, naturally
思考 sī kǎo : to reflect on, to ponder over


In [246]:
new_stuff = yabla_lookup(tt.missing(new_words))

 奥莱 卡斯特 渐近线 能够 决定 无限 怀疑 幻觉 还有 观展 其中 感官 人们 其他 直线 这样 不同 曲线 参与 哪条 愉悦 当代艺术 颜色 文森特 时间 融入 这位 看来 根据 延伸 生于 组成 视觉 应该 所以 只有 可能 不断 相同 展现 比利 才能 对称 固有 拥有 这次 现长 别人 直接 对比 参展者 近于零 冥想 彩色 比如说 觉得 二分之一


In [256]:
failed = []
for x,y in new_stuff.items():
    try:
        new_stuff[x] = clean(y)
    except:
        failed.append(x)

In [251]:
print '\ '.join(failed)

融入\ 近于零\ 还有\ 这位\ 看来\ 感官\ 文森特\ 展现\ 觉得\ 观展\ 怀疑\ 时间\ 视觉\ 卡斯特\ 颜色\ 所以\ 能够\ 根据\ 可能\ 冥想\ 只有\ 其他\ 参展者\ 人们\ 决定\ 彩色\ 不断\ 参与\ 其中\ 二分之一\ 比如说\ 比利\ 延伸\ 对称\ 应该\ 奥莱\ 哪条\ 组成\ 渐近线\ 直线\ 拥有\ 固有\ 相同\ 生于\ 才能\ 不同\ 曲线\ 现长\ 这次\ 无限\ 当代艺术\ 别人\ 幻觉\ 直接\ 对比\ 这样\ 愉悦


In [522]:
tt.update(new_stuff)

In [425]:
tt.dump()
tt = MyWords()
tt.load()

In [367]:
print '\n'.join( tt.missing(tags2))

蒋兆和
于洋
苦茶
生民
乡关
家国


In [267]:
doc = Document('/Users/macbuse/Downloads/ZHU_TRANSLATIONS/holler_CN.docx')

In [293]:
for i, pp in enumerate(doc.paragraphs[:6]):
    if not pp.text:  print i
    print pp.text

你早期的工作主要是针对农乡科学，直到1994年你一直在从事昆虫研究员的工作，专门从事的研究是昆虫如何通过嗅觉进行彼此沟通。那么是什么时候你开始转向艺术工作，导致你转变的原因是什么？
1

在早期的艺术作品中，你显示对数学和几何方面浓厚兴趣，大量作品探讨对现实世界分割的方法，在这段时间的创作对后来的空间作品有哪些方面的铺垫？
3

4

在2006年你创作了一些大型的公共性艺术空间，像《实验场》、《游乐场》、《飞行器》能否介绍一下这三件作品的思路。事实上你在对出现工业革命之后平民体验科学乐趣的方式做了一些有意识的改变，当然我们也会思考这些器材产生的乐趣本身的目的性以及也会从控制论的角度来思考这系列作品，事实上这些作品将人类置于一种封闭的快感过程当中，将主体的经验做有意识的隔离和切分，当然这些器材出现在第一次和第二次工业革命，而并未涉及到微电子时代人类对自身控制的形式，那么你是怎样看待现代人在新的工业机制中所显示的状态？


This is a really shitty way to make a webpage :()

In [7]:
wiki_words = MyWords(src_pkl='wiki_words.pkl')

In [46]:
len(wiki_words.items())
wiki_words.dump()

In [18]:
for x,y in wiki_words.items()[:10]:
    print x, type(y[0]),'%s %s'%y

要 <type 'unicode'> yào demand,request
一切 <type 'unicode'> yīqiè det.: everything, every
謝謝 <type 'unicode'> xièxiè thank
範圍 <type 'unicode'> fànwéi demesne,latitude,shot,compass,extensity,panorama,parameter,boundary,arena,stretch,territory,radius,spectrum,scope,extension,sphere,orbit,circumscription,limits,purview,grasp,extent,region,space,limit,hemisphere,horizon,envelope,bounds,realm,terrain,tether,sphere of influence,range,circuit,domain,precinct,reach,area,length,spread,bailiwick,ambit,con
怎麼 <type 'unicode'> zěnme how
於是 <type 'unicode'> yūshì conj.: thereupon,hence,consequently,as a result
一樣 <type 'unicode'> yīyàng same,equal,similar
美國 <type 'unicode'> měiguó House_of_Representatives,the United States of America,US,U.S.A.,bench,Columbia,Yankeeland,U.S.,United States,Democrat,Yankeedom,USA,America,United States of America
系列 <type 'unicode'> xìliè succession,set,spectrum,course,train,series
所以 <type 'unicode'> suǒyǐ conj.: so,therefore,as a result
